# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [7]:
#Definition of the tables.
import pandas as pd


# Definición de las tablas y sus descripciones
data = {
    'table': ['employees', 'salary', 'studies'],  # Nombres de las tablas
    'definition': [
        'Employee information, name, position, and contact details.',
        'Salary details for each year.',
        'Educational studies, name of the institution, degree, and graduation year.'
    ]
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Imprimir el DataFrame
print(df)



       table                                         definition
0  employees  Employee information, name, position, and cont...
1     salary                      Salary details for each year.
2    studies  Educational studies, name of the institution, ...


In [8]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [9]:
print(text_tables)

employees: Employee information, name, position, and contact details.
salary: Salary details for each year.
studies: Educational studies, name of the institution, degree, and graduation year.


In [11]:

# Imprimir la cabecera del DataFrame
print(df.head())


       table                                         definition
0  employees  Employee information, name, position, and cont...
1     salary                      Salary details for each year.
2    studies  Educational studies, name of the institution, ...


In [17]:
prompt_question_tables = """
Given the following tables and their content definitions:
### Tables:
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{question}
"""

text_tables = """
employees: Contains employee information such as name, position, and contact details.
salary: Contains salary details, including annual salary and employee ID.
studies: Contains educational information, including institution name, degree, and graduation year.
"""

question = "How much do employees whose names start with the letter 'A' earn?"



In [18]:
#Creating the prompt, with the user questions and the tables definitions.
pq1 = prompt_question_tables.format(tables=text_tables, question=question)

# Llamar al modelo
response = return_OAI(pq1)

# Imprimir la respuesta
print(response)


{
    "tables": ["employees", "salary"]
}


In [19]:
{
  "tables": ["employees", "salary"]
}


{'tables': ['employees', 'salary']}

In [20]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [22]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="'Find the total salary of all employees whose names start with 'A'.'")

In [23]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salary"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

To improve the utility of GPT for SQL-related tasks:

Enhance the prompt with example queries to guide the model toward deeper understanding.
Use fine-tuned models specifically trained on SQL tasks for better results.
Pair GPT's table selection capabilities with a query generation system to automate the full pipeline.
Overall, the exercise highlighted GPT's strengths in pattern recognition and consistency but revealed its limitations in generating actionable insights for complex scenarios.

Lessons Learned:
Prompt Clarity Drives Results:

Clear and structured prompts are critical to obtaining meaningful responses. GPT excels when given concise definitions of tables and explicit instructions.
Model Consistency:

The model consistently identified relevant tables, showing it can process and relate table descriptions to user questions effectively.
Contextual Guidance is Crucial:

While the model can select tables, it struggles to go beyond the surface. Adding contextual examples or instructions, such as example SQL queries, might improve its ability to provide actionable responses.
Limitations of General Models:

GPT is not optimized for tasks requiring deep logical reasoning or query construction. It highlights the need for fine-tuning or supplementing the model with domain-specific expertise for tasks like SQL query generation.